In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

# importo los .py
import sys
sys.path.append("auxiliares")
import config
import feature_eng

from warnings import simplefilter 
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
simplefilter(action="ignore", category=pd.errors.SettingWithCopyWarning)

In [2]:
####################################################
############# Setear segun cada maquina ############
#os.chdir("C:/Users/herna/labo3_empresa3_repo/")
#os.chdir("C:/diego_tools/labo3/")
os.chdir("/home/dcastro_contacto/buckets/b1/")
####################################################

In [3]:
arch_entrada = "datasets/emp3_sellout_producto_transf.csv"
arch_salida = "datasets/emp3_sellout_producto_transf_fe.csv"

# No usa los archivos cat porque primero transforma y luego los usa

In [4]:
df_sellout = pd.read_csv(arch_entrada)
df_sellout.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34707 entries, 0 to 34706
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   product_id                 34707 non-null  int64  
 1   periodo                    34707 non-null  int64  
 2   tn                         34707 non-null  float64
 3   cero_ventas                34707 non-null  int64  
 4   cust_request_qty           34707 non-null  float64
 5   cust_request_tn            34707 non-null  float64
 6   mes                        34707 non-null  int64  
 7   producto_estrella          34707 non-null  float64
 8   plan_precios_cuidados      34707 non-null  float64
 9   meses_historia_prod        34707 non-null  int64  
 10  cat1                       34707 non-null  object 
 11  cat2                       34707 non-null  object 
 12  cat3                       34707 non-null  object 
 13  sku_size                   34707 non-null  flo

## Variable Target

In [5]:
df_sellout = feature_eng.crear_target_product(df_sellout)
df_sellout.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34707 entries, 0 to 34706
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   product_id                 34707 non-null  int64  
 1   periodo                    34707 non-null  int64  
 2   tn                         34707 non-null  float64
 3   cero_ventas                34707 non-null  int64  
 4   cust_request_qty           34707 non-null  float64
 5   cust_request_tn            34707 non-null  float64
 6   mes                        34707 non-null  int64  
 7   producto_estrella          34707 non-null  float64
 8   plan_precios_cuidados      34707 non-null  float64
 9   meses_historia_prod        34707 non-null  int64  
 10  cat1                       34707 non-null  object 
 11  cat2                       34707 non-null  object 
 12  cat3                       34707 non-null  object 
 13  sku_size                   34707 non-null  flo

## Nuevos Features

In [6]:
df_sellout = feature_eng.crear_nuevos_features(df_sellout)
df_sellout.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34707 entries, 0 to 34706
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   product_id                 34707 non-null  int64  
 1   periodo                    34707 non-null  int64  
 2   tn                         34707 non-null  float64
 3   cero_ventas                34707 non-null  int64  
 4   cust_request_qty           34707 non-null  float64
 5   cust_request_tn            34707 non-null  float64
 6   mes                        34707 non-null  int64  
 7   producto_estrella          34707 non-null  float64
 8   plan_precios_cuidados      34707 non-null  float64
 9   meses_historia_prod        34707 non-null  int64  
 10  cat1                       34707 non-null  object 
 11  cat2                       34707 non-null  object 
 12  cat3                       34707 non-null  object 
 13  sku_size                   34707 non-null  flo

In [7]:
df_sellout = feature_eng.crear_features_temporales("product_id",df_sellout, config.NUM_LAGS_PARAM,config.FAMILIA_FEATURES_TEMP_PARAM)

#En vez de levantar los archivos catX, los construyo con tn ya transformado
df_cat1 = df_sellout.groupby(["cat1","periodo"])["tn"].sum().reset_index()
df_cat1 = df_cat1.rename(columns={"tn":"cat1_tn"})
df_cat1 = feature_eng.crear_features_temporales("cat1",df_cat1,config.NUM_LAGS_PARAM,config.FAMILIA_FEATURES_TEMP_PARAM)

df_cat2 = df_sellout.groupby(["cat2","periodo"])["tn"].sum().reset_index()
df_cat2 = df_cat2.rename(columns={"tn":"cat2_tn"})
df_cat2 = feature_eng.crear_features_temporales("cat2",df_cat2,config.NUM_LAGS_PARAM,config.FAMILIA_FEATURES_TEMP_PARAM)

df_cat3 = df_sellout.groupby(["cat3","periodo"])["tn"].sum().reset_index()
df_cat3 = df_cat3.rename(columns={"tn":"cat3_tn"})
df_cat3 = feature_eng.crear_features_temporales("cat3",df_cat3,config.NUM_LAGS_PARAM,config.FAMILIA_FEATURES_TEMP_PARAM)


print(len(df_sellout))
if "cat1" in config.AMPLIA_FEATURES_PARAM:
    df_sellout = df_sellout.merge(df_cat1,how="left",on=["cat1","periodo"])

if "cat2" in config.AMPLIA_FEATURES_PARAM:
    df_sellout = df_sellout.merge(df_cat2,how="left",on=["cat2","periodo"])

if "cat3" in config.AMPLIA_FEATURES_PARAM:
    df_sellout = df_sellout.merge(df_cat3,how="left",on=["cat3","periodo"])
print(len(df_sellout))

34707
34707


In [8]:
for col in df_sellout.columns:
    print(col)

product_id
periodo
tn
cero_ventas
cust_request_qty
cust_request_tn
mes
producto_estrella
plan_precios_cuidados
meses_historia_prod
cat1
cat2
cat3
sku_size
temp_media
temp_min_media
temp_max_media
IPC
promedio_mens_dolar_venta
catastrofe
tn_mas_2
dif_cust_request_tn
dif_cust_request_tn_porc
product_id_tn_lag_1
product_id_tn_delta_lag_1
product_id_tn_rolling_std_1
product_id_tn_rolling_mean_1
product_id_tn_rolling_sum_1
product_id_tn_bollinger_band_upper_1
product_id_tn_bollinger_band_lower_1
product_id_tn_lag_2
product_id_tn_delta_lag_2
product_id_tn_rolling_std_2
product_id_tn_rolling_mean_2
product_id_tn_rolling_sum_2
product_id_tn_bollinger_band_upper_2
product_id_tn_bollinger_band_lower_2
product_id_tn_lag_3
product_id_tn_delta_lag_3
product_id_tn_rolling_std_3
product_id_tn_rolling_mean_3
product_id_tn_rolling_sum_3
product_id_tn_bollinger_band_upper_3
product_id_tn_bollinger_band_lower_3
product_id_tn_lag_4
product_id_tn_delta_lag_4
product_id_tn_rolling_std_4
product_id_tn_rollin

In [9]:
df_sellout.head(5)

,product_id,periodo,tn,cero_ventas,cust_request_qty,cust_request_tn,mes,producto_estrella,plan_precios_cuidados,meses_historia_prod,...,cat1_tn_rolling_sum_14,cat1_tn_bollinger_band_upper_14,cat1_tn_bollinger_band_lower_14,cat1_tn_lag_15,cat1_tn_delta_lag_15,cat1_tn_rolling_std_15,cat1_tn_rolling_mean_15,cat1_tn_rolling_sum_15,cat1_tn_bollinger_band_upper_15,cat1_tn_bollinger_band_lower_15
0,20001,201701,-1.444241,0,479.0,-1.434763,1,1.0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20001,201702,-1.882875,0,432.0,-1.768352,2,1.0,0.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20001,201703,-0.262032,0,509.0,-0.174183,3,1.0,0.0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20001,201704,-1.010633,0,279.0,-0.808620,4,1.0,0.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20001,201705,0.375743,0,701.0,0.531264,5,1.0,0.0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_sellout.to_csv(arch_salida, index=False)

In [11]:
print(len(df_sellout))

34707
